<a href="https://colab.research.google.com/github/dai-ichiro/AnimateDiff_Multi-ControlNet_IP-Adapter/blob/main/shunt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ver.1.9
# 必要なライブラリをインストールします
!pip install python-docx -q
!apt-get -qq install -y fonts-liberation -q

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GroupKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, RocCurveDisplay
from sklearn.feature_selection import RFECV
from scipy.stats import mannwhitneyu, chi2_contingency
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
from docx import Document
from pathlib import Path
import warnings
import re

# --- グローバル設定 ---
warnings.filterwarnings('ignore')
# グラフの見た目を設定
plt.rcParams['font.family'] = 'Liberation Sans'
plt.rcParams['font.weight'] = 'bold'
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['axes.titleweight'] = 'bold'
plt.rcParams['lines.linewidth'] = 2.5

# --- 設定項目 ---
# パス設定
DRIVE_BASE_PATH = Path("/content/drive/MyDrive/wav_files")
INPUT_EXCEL_PATH = DRIVE_BASE_PATH / "features_with_fv_and_labels.xlsx"
OUTPUT_DIR = Path("./analysis_results")

# 解析パラメータ設定
TARGET_COLUMN_FOR_LABELING = 'FV'
GROUP_THRESHOLD = 400
TARGET_VARIABLE = f'{TARGET_COLUMN_FOR_LABELING}>={GROUP_THRESHOLD}'
TARGET_LABEL_NAME = 'FV'
N_FEATURES_TO_SELECT = 5
N_SPLITS_CV = 5
N_BOOTSTRAPS_CI = 1000
# 最終評価用のデータ分割を固定するための乱数シード
UNIFIED_RANDOM_STATE = 252


# --- 関数定義 ---

def load_and_preprocess_data(file_path: Path) -> pd.DataFrame:
    """Excelファイルを読み込み、目的変数を作成する"""
    if not file_path.exists():
        raise FileNotFoundError(f"指定されたファイルが見つかりません: {file_path}")
    df = pd.read_excel(file_path)
    print(f"'{file_path.name}' から臨床データを読み込みました。")
    df[TARGET_VARIABLE] = (df[TARGET_COLUMN_FOR_LABELING] >= GROUP_THRESHOLD).astype(int)
    print(f"'{TARGET_COLUMN_FOR_LABELING}'値から2値分類の正解ラベル '{TARGET_VARIABLE}' を自動生成しました。")
    print(f"n={len(df)}のデータで解析を実行します。")
    return df

def create_patient_characteristics_table(df: pd.DataFrame) -> pd.DataFrame:
    """Table 1: 患者背景テーブルを作成する"""
    print("\n--- Table 1: Patient Characteristicsを作成中 ---")
    df_patients_only = df.drop_duplicates(subset=['Pt No'])
    print(f"患者単位でのユニーク化: {len(df)}音源 → {len(df_patients_only)}患者")
    group0 = df_patients_only[df_patients_only[TARGET_VARIABLE] == 0]
    group1 = df_patients_only[df_patients_only[TARGET_VARIABLE] == 1]

    table1_data = []
    continuous_vars = ['age', 'BW', 'FV', 'RI', 'RA diameter', 'shunt diameter']
    for var in continuous_vars:
        if var in df.columns:
            mean0, std0 = group0[var].mean(), group0[var].std()
            mean1, std1 = group1[var].mean(), group1[var].std()
            _, p_val = mannwhitneyu(group0[var].dropna(), group1[var].dropna())
            table1_data.append({
                'Characteristic': var,
                f'{TARGET_LABEL_NAME} < {GROUP_THRESHOLD} (n={len(group0)})': f"{mean0:.1f} ± {std0:.1f}",
                f'{TARGET_LABEL_NAME} >= {GROUP_THRESHOLD} (n={len(group1)})': f"{mean1:.1f} ± {std1:.1f}",
                'P-Value': f"{p_val:.2f}" if p_val >= 0.01 else "<0.01"
            })

    categorical_vars = {'men': 'Male Sex (n, %)', 'arterial calcification': 'Arterial Calcification (n, %)', 'DM': 'DM (n, %)', 'HTN': 'HTN (n, %)', 'Heart disease': 'Heart Disease (n, %)'}
    for var, name in categorical_vars.items():
        if var in df_patients_only.columns:
            df_patients_only[var + '_numeric'] = df_patients_only[var].astype(str).str.lower().map({'y': 1, 'n': 0, 'male': 1, 'female': 0, '1': 1, '0': 0, '1.0': 1, '0.0': 0}).fillna(0)
            crosstab = pd.crosstab(df_patients_only[var + '_numeric'], df_patients_only[TARGET_VARIABLE])
            _, p_val, _, _ = chi2_contingency(crosstab)
            n0_positive = crosstab.loc[1, 0] if 1 in crosstab.index and 0 in crosstab.columns else 0
            n1_positive = crosstab.loc[1, 1] if 1 in crosstab.index and 1 in crosstab.columns else 0
            table1_data.append({
                'Characteristic': name,
                f'{TARGET_LABEL_NAME} < {GROUP_THRESHOLD} (n={len(group0)})': f"{n0_positive} ({n0_positive/len(group0)*100:.1f})",
                f'{TARGET_LABEL_NAME} >= {GROUP_THRESHOLD} (n={len(group1)})': f"{n1_positive} ({n1_positive/len(group1)*100:.1f})",
                'P-Value': f"{p_val:.2f}" if p_val >= 0.01 else "<0.01"
            })
    return pd.DataFrame(table1_data)

def select_features(X_df: pd.DataFrame, y: pd.Series, groups: pd.Series, feature_list: list) -> pd.DataFrame:
    """RFECVとRFのハイブリッド法で特徴量選択を行う"""
    if len(feature_list) <= N_FEATURES_TO_SELECT:
        print(f"-> 開始時の特徴量が{len(feature_list)}個のため、特徴量選択をスキップします。")
        return X_df[feature_list]

    X_subset = X_df[feature_list]
    model_for_select = RandomForestClassifier(random_state=42, class_weight='balanced')
    cv_selector = GroupKFold(n_splits=N_SPLITS_CV)

    selector = RFECV(estimator=model_for_select, min_features_to_select=N_FEATURES_TO_SELECT, step=1, cv=cv_selector, scoring='roc_auc', n_jobs=-1)
    selector.fit(X_subset, y, groups=groups)

    X_selected_temp = X_subset.loc[:, selector.support_]

    if X_selected_temp.shape[1] > N_FEATURES_TO_SELECT:
        temp_model = RandomForestClassifier(random_state=42, class_weight='balanced').fit(X_selected_temp, y)
        feature_importances = pd.Series(temp_model.feature_importances_, index=X_selected_temp.columns)
        top_features = feature_importances.nlargest(N_FEATURES_TO_SELECT).index.tolist()
        X_selected = X_subset[top_features]
    else:
        X_selected = X_selected_temp

    print(f"-> Selected {X_selected.shape[1]} features.")
    return X_selected

def evaluate_model_with_cv(X_selected: pd.DataFrame, y: pd.Series, groups: pd.Series) -> pd.DataFrame:
    """患者単位のクロスバリデーションでモデルの性能指標を評価する"""
    model_for_eval = RandomForestClassifier(random_state=42, class_weight='balanced')
    group_kfold = GroupKFold(n_splits=N_SPLITS_CV)
    scores = {'auc': [], 'accuracy': [], 'sensitivity': [], 'specificity': [], 'f1_score': []}

    for train_idx, test_idx in group_kfold.split(X_selected, y, groups):
        X_train_cv, X_test_cv = X_selected.iloc[train_idx], X_selected.iloc[test_idx]
        y_train_cv, y_test_cv = y.iloc[train_idx], y.iloc[test_idx]

        model_for_eval.fit(X_train_cv, y_train_cv)
        y_pred_proba = model_for_eval.predict_proba(X_test_cv)[:, 1]
        y_pred = model_for_eval.predict(X_test_cv)

        if len(np.unique(y_test_cv)) > 1:
            scores['auc'].append(roc_auc_score(y_test_cv, y_pred_proba))

        report = classification_report(y_test_cv, y_pred, output_dict=True, zero_division=0)
        scores['accuracy'].append(report['accuracy'])
        scores['sensitivity'].append(report['1']['recall'])
        scores['specificity'].append(report['0']['recall'])
        scores['f1_score'].append(report['weighted avg']['f1-score'])

    scores_df = pd.DataFrame(scores)
    return scores_df.agg(['mean', 'std']).T

def calculate_bootstrap_ci(y_test: pd.Series, y_pred_proba_test: np.ndarray, test_patient_ids: pd.Series) -> tuple:
    """患者レベルのブートストラップ法でAUCの95%信頼区間を計算する"""
    unique_patients_in_test = test_patient_ids.unique()
    n_patients = len(unique_patients_in_test)
    bootstrapped_scores = []
    np.random.seed(UNIFIED_RANDOM_STATE) # ブートストラップの再現性を確保

    for i in range(N_BOOTSTRAPS_CI):
        boot_patient_sample = np.random.choice(
            unique_patients_in_test,
            size=n_patients,
            replace=True
        )
        boot_indices = []
        for patient_id in boot_patient_sample:
            indices_for_patient = test_patient_ids[test_patient_ids == patient_id].index
            boot_indices.extend(indices_for_patient)

        y_test_boot = y_test.loc[boot_indices]
        integer_indices = y_test.index.get_indexer(boot_indices)
        y_pred_proba_boot = y_pred_proba_test[integer_indices]

        if len(np.unique(y_test_boot)) < 2:
            continue
        score = roc_auc_score(y_test_boot, y_pred_proba_boot)
        bootstrapped_scores.append(score)

    if not bootstrapped_scores:
        return np.nan, np.nan
    return np.percentile(bootstrapped_scores, 2.5), np.percentile(bootstrapped_scores, 97.5)

def save_tables_to_word(table1_df: pd.DataFrame, table2_df: pd.DataFrame, table3_df: pd.DataFrame, output_file: Path):
    """3つのテーブルをWordファイルに保存する"""
    document = Document()
    document.add_heading(f'Table 1: Patient Characteristics ({TARGET_VARIABLE})', level=1)
    t1 = document.add_table(table1_df.shape[0]+1, table1_df.shape[1], style='Table Grid')
    for j in range(table1_df.shape[1]): t1.cell(0,j).text = table1_df.columns[j]
    for i in range(table1_df.shape[0]):
        for j in range(table1_df.shape[1]): t1.cell(i+1,j).text = str(table1_df.values[i,j])
    document.add_page_break()

    document.add_heading(f'Table 2: Model Performance Summary ({TARGET_VARIABLE})', level=1)
    t2 = document.add_table(table2_df.shape[0]+1, table2_df.shape[1], style='Table Grid')
    for j in range(table2_df.shape[1]): t2.cell(0,j).text = table2_df.columns[j]
    for i in range(table2_df.shape[0]):
        for j in range(table2_df.shape[1]): t2.cell(i+1,j).text = str(table2_df.values[i,j])
    document.add_page_break()

    document.add_heading(f'Table 3: Top 5 Important Features per Model ({TARGET_VARIABLE})', level=1)
    table3_for_word = table3_df.reset_index().rename(columns={'index': 'Feature'})
    t3 = document.add_table(table3_for_word.shape[0]+1, table3_for_word.shape[1], style='Table Grid')
    for j in range(table3_for_word.shape[1]): t3.cell(0,j).text = table3_for_word.columns[j]
    for i in range(table3_for_word.shape[0]):
        for j in range(table3_for_word.shape[1]):
            cell_value = table3_for_word.values[i,j]
            t3.cell(i+1,j).text = f"{cell_value:.4f}" if isinstance(cell_value, (float, np.floating)) else str(cell_value)
    document.save(output_file)
    print(f"\nテーブルを '{output_file}' に保存しました。")

def plot_roc_curves(final_results: dict, table2_df: pd.DataFrame, output_file: Path):
    """Figure 1: ROC曲線を描画・保存する"""
    plt.figure(figsize=(10, 8)); ax = plt.gca()
    linestyles = ['-', '--', ':']
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c']

    for i, (model_name, result) in enumerate(final_results.items()):
        cv_auc_mean = table2_df.loc[table2_df['Model'] == model_name, 'AUC'].values[0].split(' ±')[0]
        label = f"{model_name} (AUC = {float(cv_auc_mean):.3f})"
        RocCurveDisplay.from_estimator(
            result['model'], result['X_test'], result['y_test'],
            name=label, ax=ax, linestyle=linestyles[i % len(linestyles)], color=colors[i % len(colors)]
        )
    plt.plot([0, 1], [0, 1], color='black', lw=1, linestyle='--')
    plt.title('Figure 1: ROC Curves with 5-fold Cross-Validation Results', fontsize=16, weight='bold')
    plt.xlabel('False Positive Rate (1 - Specificity)', fontsize=12, weight='bold')
    plt.ylabel('True Positive Rate (Sensitivity)', fontsize=12, weight='bold')
    plt.grid(linestyle=':', alpha=0.6)
    plt.legend(fontsize=12)
    plt.tight_layout()
    plt.savefig(output_file, format='svg', bbox_inches='tight')
    plt.show()
    print(f"Figure 1を '{output_file}' に保存しました。")

def plot_feature_boxplots(df_orig: pd.DataFrame, table3_df: pd.DataFrame, output_dir: Path) -> list:
    """Figure 2: 特徴量の箱ひげ図を描画・保存する"""
    if 'A-E (Integrated)' not in table3_df.columns:
        return []

    df_orig['Group'] = np.where(df_orig[TARGET_VARIABLE] == 0, f'{TARGET_LABEL_NAME} < {GROUP_THRESHOLD}', f'{TARGET_LABEL_NAME} >= {GROUP_THRESHOLD}')
    features_to_plot = table3_df['A-E (Integrated)'].dropna().head(3).index.tolist()
    figure2_files = []
    print(f"\n--- Top 3 features for boxplot: {features_to_plot} ---")

    for i, feature in enumerate(features_to_plot):
        plt.figure(figsize=(5, 5))
        sns.boxplot(x='Group', y=feature, data=df_orig, palette='viridis')
        sns.stripplot(x='Group', y=feature, data=df_orig, color=".25", size=6)
        plt.title(f'Figure 2-{i+1}: Distribution of {feature}')
        plt.ylabel(feature)
        plt.xlabel('')
        plt.tight_layout()
        output_file = output_dir / f'Figure2_BoxPlot_{TARGET_LABEL_NAME}_{feature}_revised.svg'
        plt.savefig(output_file, format='svg', bbox_inches='tight')
        plt.show()
        print(f"Figure 2-{i+1}を '{output_file}' に保存しました。")
        figure2_files.append(output_file)
    return figure2_files

def download_files_in_colab(files_to_download: list):
    """生成されたファイルをColab環境でダウンロードする"""
    print("\n--- 生成されたファイルのダウンロードを開始します ---")
    try:
        from google.colab import files
        for f in files_to_download:
            files.download(str(f))
    except (ImportError, NameError):
        print("ダウンロードはGoogle Colab環境でのみ有効です。")
        print("ファイルは左側のファイルブラウザからダウンロードできます。")

def main():
    """メインの実行関数"""
    drive.mount('/content/drive')
    OUTPUT_DIR.mkdir(exist_ok=True)

    df_orig = load_and_preprocess_data(INPUT_EXCEL_PATH)
    table1_df = create_patient_characteristics_table(df_orig)
    print(table1_df.to_string())

    df_processed = df_orig.copy()
    if 'HD(a, b, c)' in df_processed.columns:
        df_processed = pd.get_dummies(df_processed, columns=['HD(a, b, c)'], prefix='HD', drop_first=False)

    y = df_processed[TARGET_VARIABLE]
    groups = df_orig['Pt No']

    # 特徴量セットの定義
    cols_A_to_D = [col for col in df_orig.columns if re.match(r"^[A-D]\d+", col)]
    cols_E_only = [col for col in df_orig.columns if re.match(r"^E\d+", col)]
    cols_A_to_E = cols_A_to_D + cols_E_only
    model_types = {"A-D (Conventional)": cols_A_to_D, "E only (Advanced)": cols_E_only, "A-E (Integrated)": cols_A_to_E}

    # ★★★ 改善提案①：堅牢な特徴量選択（cols_to_excludeを廃止） ★★★
    # ★★★ 改善提案②：可読性向上のため、特徴量列を直接数値変換（変数Xを廃止） ★★★
    print("\n--- 特徴量として使用する列のデータ型を数値に統一 ---")
    for col in cols_A_to_E:
        df_processed[col] = pd.to_numeric(df_processed[col], errors='coerce').fillna(0)
    print("-> 完了")


    final_results = {}
    summary_data = []
    table3_data = {}

    print("\n--- 最終評価のためのデータ分割を固定 ---")
    unique_patients = df_orig['Pt No'].unique()
    patient_labels = df_orig.drop_duplicates(subset=['Pt No']).set_index('Pt No')[TARGET_VARIABLE]

    train_patients, test_patients = train_test_split(
        unique_patients,
        test_size=0.2,
        random_state=UNIFIED_RANDOM_STATE,
        stratify=patient_labels.reindex(unique_patients)
    )
    train_indices = df_orig[df_orig['Pt No'].isin(train_patients)].index
    test_indices = df_orig[df_orig['Pt No'].isin(test_patients)].index

    print(f"患者分割を固定: 訓練{len(train_patients)}患者, テスト{len(test_patients)}患者")
    print(f"音源数: 訓練{len(train_indices)}音源, テスト{len(test_indices)}音源")

    print("\n--- 特徴選択とモデル評価を開始 ---")
    for model_name, feature_list in model_types.items():
        print(f"\nProcessing model: {model_name} (Initial features: {len(feature_list)})")

        # ★★★ 改善提案②を反映：Xの代わりにdf_processedを渡す ★★★
        X_selected = select_features(df_processed, y, groups, feature_list)

        model_for_importance = RandomForestClassifier(random_state=42, class_weight='balanced').fit(X_selected, y)
        importances = pd.Series(model_for_importance.feature_importances_, index=X_selected.columns).sort_values(ascending=False)
        table3_data[model_name] = importances

        cv_performance = evaluate_model_with_cv(X_selected, y, groups)

        X_train, X_test = X_selected.loc[train_indices], X_selected.loc[test_indices]
        y_train, y_test = y.loc[train_indices], y.loc[test_indices]
        groups_test = groups.loc[test_indices]

        final_model = RandomForestClassifier(random_state=42, class_weight='balanced').fit(X_train, y_train)
        y_pred_proba_test = final_model.predict_proba(X_test)[:, 1]

        auc_ci_lower, auc_ci_upper = calculate_bootstrap_ci(y_test, y_pred_proba_test, groups_test)

        summary_data.append({
            'Model': model_name, 'Features': X_selected.shape[1],
            'AUC': f"{cv_performance.loc['auc', 'mean']:.3f} ± {cv_performance.loc['auc', 'std']:.3f}",
            'AUC (95% CI)': f"{auc_ci_lower:.3f} - {auc_ci_upper:.3f}",
            'Accuracy': f"{cv_performance.loc['accuracy', 'mean']:.3f} ± {cv_performance.loc['accuracy', 'std']:.3f}",
            'Sensitivity': f"{cv_performance.loc['sensitivity', 'mean']:.3f} ± {cv_performance.loc['sensitivity', 'std']:.3f}",
            'Specificity': f"{cv_performance.loc['specificity', 'mean']:.3f} ± {cv_performance.loc['specificity', 'std']:.3f}",
            'F1-Score': f"{cv_performance.loc['f1_score', 'mean']:.3f} ± {cv_performance.loc['f1_score', 'std']:.3f}"
        })
        final_results[model_name] = {'model': final_model, 'X_test': X_test, 'y_test': y_test}

    table2_columns = ['Model', 'Features', 'AUC', 'AUC (95% CI)', 'Accuracy', 'Sensitivity', 'Specificity', 'F1-Score']
    table2_df = pd.DataFrame(summary_data)[table2_columns]
    print("\n--- Table 2: Model Performance Summary ---")
    print(table2_df.to_string(index=False))

    table3_df = pd.DataFrame(table3_data)
    print("\n--- Table 3: Top 5 Important Features per Model ---")
    print(table3_df)

    files_to_download = []
    table_word_file = OUTPUT_DIR / f'Tables_Analysis_{TARGET_LABEL_NAME}_revised.docx'
    save_tables_to_word(table1_df, table2_df, table3_df, table_word_file)
    files_to_download.append(table_word_file)

    figure1_svg_file = OUTPUT_DIR / f'Figure1_ROC_Curve_{TARGET_LABEL_NAME}_corrected.svg'
    plot_roc_curves(final_results, table2_df, figure1_svg_file)
    files_to_download.append(figure1_svg_file)

    figure2_files = plot_feature_boxplots(df_orig, table3_df, OUTPUT_DIR)
    files_to_download.extend(figure2_files)

    download_files_in_colab(files_to_download)

if __name__ == "__main__":
    main()